<a href="https://colab.research.google.com/github/yeipi-mora/personal-projects/blob/main/arbitrage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
import gspread
from google.auth import default
from google.colab import auth

In [ ]:
class Graph:

    def __init__(self, vertices):

        self.vertices = vertices   # Total number of vertices in the graph

        self.edges = []     # Array of edges

    # Add edges: a->b with weight w

    def add_edge(self, a, b, w):

        self.edges.append([a, b, w])

    def directed_neighbors(self, u):

      neighbor = []

      for edge in self.edges:

        if u == edge[0]:

          neighbor.append(edge[1])

      return neighbor

class Bellman_Ford:

    def __init__(self, G):

        self.V = G.vertices   # Total number of vertices in the graph

        self.E = G.edges     # Array of edges

    #initialize graph

    def initialize_graph(self, s):

      self.d = [float("Inf")]*self.V

      self.d[s] = 0

    #relax procedure

    def relax(self, a, b, w):

      if self.d[a] != float("Inf") and self.d[b] >  self.d[a] + w:

        self.d[b] = self.d[a] + w

    #Bellman Ford Algorithm with vertex s as source

    def bellman_ford(self, s):

        self.initialize_graph(s)

        for _ in range(self.V - 1):

            for a, b, w in self.E:

              self.relax(a, b, w)

        for a, b, w in self.E:

            if self.d[b] > self.d[a] + w:

                #print('Graph contains negative weight cycle')

                return True

        return False

    def negative_cycle(self, G, s) -> list:

      if self.bellman_ford(s):

        #save copy of distances from s

        delta = [distance for distance in self.d]

        #new relax iteration to find vertexes in negative cycles

        for _ in range(self.V):

            for a, b, w in self.E:

              self.relax(a, b, w)

        #set to save vertx in negative cycles

        S = list()

        for v in range(self.V):

          if self.d[v] < delta[v]:

            S.append(v)

        T = set(S)

        #random choice of vertex in S

        v = random.choice(S)

        u = random.choice(list(T.intersection(set(G.directed_neighbors(v)))))

        C = [v, u]

        while u!=v:

          neighbors = G.directed_neighbors(u)

          w = random.choice(list(T.intersection(set(neighbors))))

          C.append(w)

          u = w

        #print('Negative cycles have been found')

        return C

      else: return []

class Eco:

  def __init__(self, A):

    self.G = Graph(len(A))

    for u in range(self.G.vertices):

      for v in range(self.G.vertices):

        if u!=v and (([u, v, A[u, v]] and [v, u, A[v, u]]) not in self.G.edges):

          self.G.add_edge(u, v, A[u, v])

          self.G.add_edge(v, u, A[v, u])

  def transform(self):

    new_G = self.G

    for edge in new_G.edges:

      edge[2] = (-1) * np.log(edge[2])

    return new_G

class arbitrage:

  def __init__(self) -> None:

    auth.authenticate_user()

    creds, _ = default()

    gc = gspread.authorize(creds)

    worksheet = gc.open('rates').sheet1

    self.df = pd.DataFrame.from_records(worksheet.get_all_values())

    self.prices = self.df.iloc[1:, 1:].values.astype(float)

    self.C = Bellman_Ford(Eco(self.prices).transform()).negative_cycle(Eco(self.prices).transform(), 0)

  # Calcula el factor de ganancia/perdida del arbitraje.

  def pctg(self, C: list, prices: np.array) -> float:

    pctg = 1

    if len(C) != 0:

      for i in range(len(C)-1):

        pctg *= prices[C[i], C[i+1]]

    return pctg

In [ ]:
a = arbitrage()
C = a.C
pctg = a.pctg(C,a.prices)
while pctg < 1.0005:
  a = arbitrage()
  C = a.C
  pctg = a.pctg(C,a.prices)
print(C,round((pctg - 1) * 100,4))

[3, 2, 4, 1, 4, 5, 4, 2, 5, 3] 0.2959
